In [1]:
import pandas as pd
import msticpy
from msticpy.vis.data_viewer import DataViewer
from msticpy.data.azure_data import AzureData
from IPython.display import display, HTML, Markdown
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
import ipywidgets as widgets
import azure.mgmt.resourcegraph
from config import jaelib, Auto_huntingQuery, Filter_huntingQuery
import types

msticpy.settings.refresh_config() #Local MSTICPY config Load
msticpy.init_notebook(globals())
ti_lookup = msticpy.TILookup("VirusTotal") #Load TIProviders VirusTotal

# Azure Key Vault
kVname = "jaehwansecret"
KVuri = "https://jaehwansecret.vault.azure.net/"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVuri, credential=credential)
API_secret = client.get_secret("arctest") # input your API Azure Key Vault SecretName
O365_secret = client.get_secret("cloffice") # input your O365 Azure Key Vault SecretName
OpenAI_secret = client.get_secret("myopenai") # input your OpenAI Azure Key Vault SecretName

In [2]:
# The file, config.json, was generated by the system, however, you may modify the values, or manually set the variables
tenant_id, subscription_id, resource_group, workspace_id, workspace_name, client_id = jaelib.read_config_values('./config/API_config.json')
O365_tenant_id, O365_subscription_id, O365_workspace_id, O365_client_id, O365_tenantdomain = jaelib.read_config_values1('./config/O365_config.json')

# Connect LogAnalytics Provider
qry_prov = msticpy.QueryProvider("LogAnalytics")
qry_prov.connect(msticpy.WorkspaceConfig(workspace="Default"))

# Connect AzureResourceGraph Provider
arg_prov = msticpy.QueryProvider("ResourceGraph")
arg_prov.connect(tenant_id=O365_tenant_id, client_id=O365_client_id, client_secret=O365_secret.value)

# Connect M365Defender Provider
md_prov = msticpy.QueryProvider("MDE")
md_prov.connect(tenant_id=tenant_id, client_id=client_id, client_secret=API_secret.value)

# Connect Azure Data Connector
az_prov = AzureData()
az_prov.connect(tenant_id=tenant_id, client_id=client_id, client_secret=API_secret.value)

connected
Connected
Connected.


In [3]:
from datetime import datetime, timedelta
class MyAlert:
    pass
alert = MyAlert()
alert.TimeGenerated = datetime.utcnow() - timedelta(1) # !!Office365 시간 필터는 24시간까지 제한이고, -7days 이상을 넘기면 안됩니다.  # type: ignore
q_times = msticpy.nbwidgets.QueryTime(units='day', max_before=20, before=3, max_after=1, origin_time=alert.TimeGenerated, auto_display=True)  # type: ignore

In [4]:
# 필터링한 시간동안 Microsoft Defender for Endpoint에서 탐지된 모든 경고를 파라미터로 저장합니다.
day_time = datetime.utcnow() - q_times.start
qry = f"""DeviceAlertEvents|where Timestamp > ago({day_time.days}d)|summarize count() by AlertId, Timestamp, Title, Severity, DeviceName, FileName|order by count_""" # DeviceAlertEvents 테이블은 AlertInfo 및 AlertEvidence 테이블을 대체합니다.
rst = md_prov.exec_query(query=qry)
display(HTML('''<h2>Step1: Time Range 동안 발생한 주요 보안 이벤트를 검토합니다.</h2>
<p>Defender for Endpoint로부터 Time Range 동안 발생한 모든 보안 이벤트를 검토합니다. 디바이스, 파일에 따른 상세 분석은 Step2에서 확인하시기 바랍니다.</p>'''))
display(rst)

,AlertId,Timestamp,Title,Severity,DeviceName,FileName,count_
0,ed638404515340969285_-722884377,2024-01-09 18:35:02.466554200+00:00,테스트 검색 규칙입니다.,High,eus-win-vm,77EC63BDA74BD0D0E0426DC8F8008506,2
1,ed638404083071045260_-789654730,2024-01-09 13:35:55.949660600+00:00,테스트 검색 규칙입니다.,High,test-bastion-wi,COMPONENTS,1
2,ed638393675973993186_-1902759592,2024-01-03 13:17:45.294461500+00:00,테스트 검색 규칙입니다.,High,vm-honey,VMAgentLogs.zip,1
3,ed638404515352780413_-538062830,2024-01-10 02:11:37.699791200+00:00,테스트 검색 규칙입니다.,High,win-vm,VMAgentLogs.zip,1
4,ed638403214608344923_331778187,2024-01-09 01:50:51.926808+00:00,테스트 검색 규칙입니다.,High,vm-honey,VMAgentLogs.zip,1
5,ed638393675973993186_-1902759592,2024-01-04 01:26:45.540234400+00:00,테스트 검색 규칙입니다.,High,vm-honey,VMAgentLogs.zip,1
6,ed638393675973993186_-1902759592,2024-01-04 05:30:00.446079700+00:00,테스트 검색 규칙입니다.,High,vm-honey,VMAgentLogs.zip,1
7,ed638403214608344923_331778187,2024-01-08 13:40:58.504661800+00:00,테스트 검색 규칙입니다.,High,vm-honey,VMAgentLogs.zip,1
8,ed638404515352780413_-538062830,2024-01-10 18:23:01.422647200+00:00,테스트 검색 규칙입니다.,High,win-vm,VMAgentLogs.zip,1
9,ed638403214608344923_331778187,2024-01-09 05:54:43.773727300+00:00,테스트 검색 규칙입니다.,High,vm-honey,VMAgentLogs.zip,1


In [5]:
#  필터링할 파일 이름을 선택하세요. 파일은 필터링한 디바이스로부터 검색되어야만 합니다. 그렇지 않다면 에러가 발생하므로 위 경고에서 잘 확인해야 합니다.
Seleted_Filename = [i if i else "N/A" for i in rst.FileName.values]
Seleted_Filename = [i for i in Seleted_Filename if i[0]]
Seleted_Filename = widgets.Dropdown(options = Seleted_Filename, description='FileName:')
display(Seleted_Filename)

# 필터링할 디바이스를 선택하세요.
Seleted_Device = [i if i else "N/A" for i in rst.DeviceName.values]
Seleted_Device = [i for i in Seleted_Device if i[0]]
Seleted_Device = widgets.Dropdown(options = Seleted_Device, description='DeviceName:')
display(Seleted_Device)

Dropdown(description='FileName:', options=('77EC63BDA74BD0D0E0426DC8F8008506', 'COMPONENTS', 'VMAgentLogs.zip'…

Dropdown(description='DeviceName:', options=('eus-win-vm', 'test-bastion-wi', 'vm-honey', 'win-vm', 'vm-honey'…

In [40]:
# Time Range, DeviceName, FileName을 필터링해서 파일 이벤트를 출력합니다.
qry = f"""DeviceFileEvents|where Timestamp > ago({day_time.days}d)|where DeviceName == '{Seleted_Device.value}'|where FileName == '{Seleted_Filename.value}'"""
results = md_prov.exec_query(query=qry)

if (not isinstance(results, pd.DataFrame)) or results.empty:
    print("No MDE Query Events, If you think that this is not correct, plz check the time range or FieName you are using")

shells = results['FileName']

pick_shell = widgets.Dropdown(
    options=shells,
    description="FileName",
    disabled=False
)

if isinstance(results, pd.DataFrame) and not results.empty:
    display(HTML('''<h2>Step2: FileName 관련 이벤트 정보를 분석합니다. VirusTotal을 활용하여 해당 파일의 MD5 분석결과를 출력합니다.</h2>
    <p>쿼리에서 새로운 필터링을 추가하고자 한다면 첫 줄의 '|where..' 라인을 추가하여 재실행하도록 합니다.</p>
    VirusTotal API를 통합하여 해당 파일(MD5)의 Malicious 여부를 자동으로 분석합니다. 하단의 URL 링크를 클릭하면 분석 결과 및 Certificate 상태를 자세히 확인할 수 있습니다.</p>'''))
    display(results)
else:
    md("Not relevant alerts were found in your MDE logs, try expanding your timeframe")

#InitiatingProcessMD5에 대한 VirustTotal 파일 스캔 정보를 출력합니다.
summary_InitiatingProcessMD5 = list(set(results["InitiatingProcessMD5"])) #InitiatingProcessMD5 Column 중복 제거
text = ""
for word in summary_InitiatingProcessMD5:
    text += word +""
    rst = ti_lookup.lookup_ioc(text)
    if not rst.empty:
        display(ti_lookup.browse_results(rst, severities='all'))


,Timestamp,DeviceId,DeviceName,ActionType,FileName,FolderPath,SHA1,SHA256,MD5,FileOriginUrl,FileOriginReferrerUrl,FileOriginIP,PreviousFolderPath,PreviousFileName,FileSize,InitiatingProcessAccountDomain,InitiatingProcessAccountName,InitiatingProcessAccountSid,InitiatingProcessAccountUpn,InitiatingProcessAccountObjectId,InitiatingProcessMD5,InitiatingProcessSHA1,InitiatingProcessSHA256,InitiatingProcessFolderPath,InitiatingProcessFileName,...,InitiatingProcessVersionInfoProductVersion,InitiatingProcessVersionInfoInternalFileName,InitiatingProcessVersionInfoOriginalFileName,InitiatingProcessVersionInfoFileDescription,InitiatingProcessId,InitiatingProcessCommandLine,InitiatingProcessCreationTime,InitiatingProcessIntegrityLevel,InitiatingProcessTokenElevation,InitiatingProcessParentId,InitiatingProcessParentFileName,InitiatingProcessParentCreationTime,RequestProtocol,RequestSourceIP,RequestSourcePort,RequestAccountName,RequestAccountDomain,RequestAccountSid,ShareName,SensitivityLabel,SensitivitySubLabel,IsAzureInfoProtectionApplied,ReportId,AppGuardContainerId,AdditionalFields
0,2023-12-28 05:35:32.112834300+00:00,f89ab196051c4c70523ee022d8c2b63fcb05273e,vm-honey,FileCreated,VMAgentLogs.zip,D:\CollectGuestLogsTemp\VMAgentLogs.zip,,,,,,,,,0,nt authority,system,S-1-5-18,,,c7884c8286969cb2e6cf7322f1ca82bc,433aaa504366752b244aa3b9748edaa00fe90b94,1f581c8085e0844c951c39699460fa8f6fd51ab0b719134ea9389c872c5708c4,c:\windowsazure\packages\collectguestlogs.exe,CollectGuestLogs.exe,...,3.0.02370.28,CollectGuestLogs.exe,CollectGuestLogs.exe,CollectGuestLogs,1952,"""CollectGuestLogs.exe"" -Mode:ga -FileName:D:\CollectGuestLogsTemp\VMAgentLogs.zip",2023-12-28 05:35:17.767696+00:00,System,TokenElevationTypeDefault,3068,WaAppAgent.exe,2023-12-28 05:34:45.591778300+00:00,Local,,None,SYSTEM,NT AUTHORITY,S-1-5-18,,,,None,1652,,"{""FileType"":""Unknown""}"
1,2023-12-28 06:38:10.538766800+00:00,f89ab196051c4c70523ee022d8c2b63fcb05273e,vm-honey,FileModified,VMAgentLogs.zip,D:\CollectGuestLogsTemp\VMAgentLogs.zip,8b272d539f5f0e6c08f786993661cce4bd764fe4,782162a445c5ba932874c9efaec1544f9cec2328b9ed956fbedaf47059936f62,20b712b0ccbb370c8df54fc14f7ff986,,,,,,6636545,nt authority,system,S-1-5-18,,,c7884c8286969cb2e6cf7322f1ca82bc,433aaa504366752b244aa3b9748edaa00fe90b94,1f581c8085e0844c951c39699460fa8f6fd51ab0b719134ea9389c872c5708c4,c:\windowsazure\packages\collectguestlogs.exe,CollectGuestLogs.exe,...,3.0.02370.28,CollectGuestLogs.exe,CollectGuestLogs.exe,CollectGuestLogs,7684,"""CollectGuestLogs.exe"" -Mode:ga -FileName:D:\CollectGuestLogsTemp\VMAgentLogs.zip",2023-12-28 06:37:11.718195100+00:00,System,TokenElevationTypeDefault,3068,WaAppAgent.exe,2023-12-28 05:34:45.591778300+00:00,Local,,None,SYSTEM,NT AUTHORITY,S-1-5-18,,,,None,6587,,"{""FileType"":""Zip""}"
2,2023-12-28 07:39:08.647969800+00:00,f89ab196051c4c70523ee022d8c2b63fcb05273e,vm-honey,FileModified,VMAgentLogs.zip,D:\CollectGuestLogsTemp\VMAgentLogs.zip,a8aa2091eda6606236fda777e2b9ccfbd0074744,e95d6bc63c8a9de1fc39816bdc5f3aed9bf9951b2e986c6e4cf9f9d70b8b661d,7b45fa54e37ae326613f7d5b0a75732f,,,,,,6652377,nt authority,system,S-1-5-18,,,c7884c8286969cb2e6cf7322f1ca82bc,433aaa504366752b244aa3b9748edaa00fe90b94,1f581c8085e0844c951c39699460fa8f6fd51ab0b719134ea9389c872c5708c4,c:\windowsazure\packages\collectguestlogs.exe,CollectGuestLogs.exe,...,3.0.02370.28,CollectGuestLogs.exe,CollectGuestLogs.exe,CollectGuestLogs,6528,"""CollectGuestLogs.exe"" -Mode:ga -FileName:D:\CollectGuestLogsTemp\VMAgentLogs.zip",2023-12-28 07:38:12.244225600+00:00,System,TokenElevationTypeDefault,3068,WaAppAgent.exe,2023-12-28 05:34:45.591778300+00:00,Local,,None,SYSTEM,NT AUTHORITY,S-1-5-18,,,,None,7259,,"{""FileType"":""Zip""}"
3,2023-12-28 08:40:05.592693900+00:00,f89ab196051c4c70523ee022d8c2b63fcb05273e,vm-honey,FileModified,VMAgentLogs.zip,D:\CollectGuestLogsTemp\VMAgentLogs.zip,a163714143eae5c1085080fe0288ae1e382eb239,9205a29cccd4e0857ad109d6b04bfabe8bfc520e10a4459ba8daed453e8e9ac6,5392d887e733227a0e581e8a91583476,,,

Observables processed:   0%|          | 0/1 [00:00<?, ?obs/s]

In [7]:
# Time Range, DeviceName을 필터링해서 디바이스 프로세스 이벤트를 출력합니다.
qry = f"""DeviceProcessEvents|where Timestamp > ago({day_time.days}d)|where DeviceName == "{Seleted_Device.value}"|where not(InitiatingProcessParentFileName in ('MsSense.exe', 'WindowsAzureGuestAgent.exe', 'SenseIR.exe', 'SenseIR.exe'))|extend TimeGenerated = Timestamp"""
results = md_prov.exec_query(query=qry)

if (not isinstance(results, pd.DataFrame)) or results.empty:
    print("No MDE Query Events, If you think that this is not correct, plz check the time range or FieName you are using")

shells = results['FileName']

pick_shell = widgets.Dropdown(
    options=shells,
    description="FileName",
    disabled=False
)

if isinstance(results, pd.DataFrame) and not results.empty:
    display(HTML('''<h2>Step3: 선택한 Device에서 Process 이벤트를 분석합니다.</h2>
    <p>쿼리에서 새로운 필터링을 추가하고자 한다면 첫 줄의 '|where..' 라인을 추가하여 재실행하도록 합니다. 또는 False Negative로 확인된 ProcessFileName이 출력되지 않도록 '|where not..' 옵션에 추가하십시오.</p>
    <p>ProcessCommandLine 칼럼을 검토하십시오. 필요하다면 Sep1, 2로 되돌아가 재분석을 수행하도록 합니다.</p>'''))
    display(results)
else:
    md("Not relevant alerts were found in your MDE logs, try expanding your timeframe")

# MSTICPy Process Tree로 표시합니다.
results.mp_timeline.plot(source_columns=["AccountName", "FileName", "ProcessCommandLine"], group_by="AccountName")
results.query("AccountName != 'system'").mp_process_tree.plot(legend_col="AccountName", show_table=True) 


,Timestamp,DeviceId,DeviceName,ActionType,FileName,FolderPath,SHA1,SHA256,MD5,FileSize,ProcessVersionInfoCompanyName,ProcessVersionInfoProductName,ProcessVersionInfoProductVersion,ProcessVersionInfoInternalFileName,ProcessVersionInfoOriginalFileName,ProcessVersionInfoFileDescription,ProcessId,ProcessCommandLine,ProcessIntegrityLevel,ProcessTokenElevation,ProcessCreationTime,AccountDomain,AccountName,AccountSid,AccountUpn,...,InitiatingProcessTokenElevation,InitiatingProcessSHA1,InitiatingProcessSHA256,InitiatingProcessMD5,InitiatingProcessFileName,InitiatingProcessFileSize,InitiatingProcessVersionInfoCompanyName,InitiatingProcessVersionInfoProductName,InitiatingProcessVersionInfoProductVersion,InitiatingProcessVersionInfoInternalFileName,InitiatingProcessVersionInfoOriginalFileName,InitiatingProcessVersionInfoFileDescription,InitiatingProcessId,InitiatingProcessCommandLine,InitiatingProcessCreationTime,InitiatingProcessFolderPath,InitiatingProcessParentId,InitiatingProcessParentFileName,InitiatingProcessParentCreationTime,InitiatingProcessSignerType,InitiatingProcessSignatureStatus,ReportId,AppGuardContainerId,AdditionalFields,TimeGenerated
0,2023-12-28 05:34:51.386428600+00:00,f89ab196051c4c70523ee022d8c2b63fcb05273e,vm-honey,ProcessCreated,wevtutil.exe,C:\Windows\System32\wevtutil.exe,ecd8921d6a8baabae23b47b126c5755b6fa11cd0,135c463428a44e91a3d8c4f01529514ddac4ef7f52c04ae42a2cf98653083408,81d8232cb4839c0c67449b1d418855f4,259584.0,Microsoft Corporation,Microsoft® Windows® Operating System,10.0.17763.3532,wevtutil.exe,wevtutil.exe,Eventing Command Line Utility,2588,"wevtutil.exe install-manifest ""C:\ProgramData\Microsoft\Windows Defender\platform\4.18.23110.3-0...",System,TokenElevationTypeDefault,2023-12-28 05:34:51.384440500+00:00,nt authority,system,S-1-5-18,,...,TokenElevationTypeDefault,8e51703b8b91287b0564c7684bc476e2d7888eff,7acd545afee1c8c9210b4bb6aa73d93b35d6aad2e330ec94bbcd2732dc8008c0,6eb45e2626f7d47cb0f491a1f2ef7e3e,MsMpEng.exe,133592.0,Microsoft Corporation,Microsoft® Windows® Operating System,4.18.23110.3,MsMpEng.exe,MsMpEng.exe,Antimalware Service Executable,3204,"""MsMpEng.exe""",2023-12-28 05:34:45.786828900+00:00,c:\programdata\microsoft\windows defender\platform\4.18.23110.3-0\msmpeng.exe,748,services.exe,2023-12-28 05:34:39.021204300+00:00,OsVendor,Valid,341,,,2023-12-28 05:34:51.386428600+00:00
1,2023-12-28 05:34:51.394894800+00:00,f89ab196051c4c70523ee022d8c2b63fcb05273e,vm-honey,ProcessCreated,conhost.exe,C:\Windows\System32\conhost.exe,c09aa822157967c4a27728f5c33d0226c2889954,ac68880b09834f3f1b12eea5966f42ad695711e04c34a577596c1578997f90e8,c15e2496b1eca76f4b09b109dab10fc3,823296.0,Microsoft Corporation,Microsoft® Windows® Operating System,10.0.17763.4840,ConHost,CONHOST.EXE,Console Window Host,2720,conhost.exe 0xffffffff -ForceV1,System,TokenElevationTypeDefault,2023-12-28 05:34:51.391093700+00:00,nt authority,system,S-1-5-18,,...,TokenElevationTypeDefault,ecd8921d6a8baabae23b47b126c5755b6fa11cd0,135c463428a44e91a3d8c4f01529514ddac4ef7f52c04ae42a2cf98653083408,81d8232cb4839c0c67449b1d418855f4,wevtutil.exe,259584.0,Microsoft Corporation,Microsoft® Windows® Operating System,10.0.17763.3532,wevtutil.exe,wevtutil.exe,Eventing Command Line Utility,2588,"wevtutil.exe install-manifest ""C:\ProgramData\Microsoft\Windows Defender\platform\4.18.23110.3-0...",2023-12-28 05:34:51.384440500+00:00,c:\windows\system32\wevtutil.exe,3204,MsMpEng.exe,2023-12-28 05:34:45.786828900+00:00,OsVendor,Valid,342,,,2023-12-28 05:34:51.394894800+00:00
2,2023-12-28 05:34:51.572078700+00:00,f89ab196051c4c70523ee022d8c2b63fcb05273e,vm-honey,ProcessCreated,wevtutil.exe,C:\Windows\System32\wevtutil.exe,ecd8921d6a8baabae23b47b126c5755b6fa11cd0,135c463428a44e91a3d8c4f01529514ddac4ef7f52c04ae42a2cf98653083408,81d8232cb4839c0c67449b1d418855f4,259584.0,Microsoft Corporation,Microsoft® Windows® Operating System,10.0.17763.3532,wevtutil.exe,wevtutil.exe,Eventing Command Line Utility,3776,"wevtutil.exe uninstall-manifest ""C:\ProgramData\Micr

Loading BokehJS ...

d:\codetest\.venv\Lib\site-packages\msticpy\transform\proc_tree_build_mde.py:343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["ProcessCreationTime"] = data[schema.time_stamp].fillna(_UNK_TIME)
d:\codetest\.venv\Lib\site-packages\msticpy\transform\proc_tree_build_mde.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["InitiatingProcessCreationTime"] = data.InitiatingProcessCreationTime.fillna(
d:\codetest\.venv\Lib\site-packages\msticpy\transform\proc_tree_build_mde.py:347: SettingWithCopyWarn

Loading BokehJS ...

(figure(id='p1344', ...), Column(id='p1524', ...))

In [8]:
qry1 = Filter_huntingQuery.get_powershell_execution_policy(Seleted_Device.value)
qry2 = Filter_huntingQuery.get_commandline_password(Seleted_Device.value)
qry3 = Filter_huntingQuery.get_webshell_detection(Seleted_Device.value)
cmdline_qry = []
cmdline_qry = qry1, qry2, qry3

for i in range(3):
    rst2 = md_prov.exec_query(query=cmdline_qry[i])
    if (not isinstance(rst2, pd.DataFrame)) or rst2.empty:
        print("No MDE Query Events, If you think that this is not correct, plz check the time range or FieName you are using")
    elif isinstance(rst2, pd.DataFrame) and not rst2.empty:
        display(HTML('''<h2>Step4: Device의 ProcessEvent로부터 비정상 Commandline을 검사하였습니다.</h2>
        <p>아래는 get_commandline_password, get_powershell_execution_policy, get_webshell_detection 3개의 쿼리가 각각 실행된 결과입니다.</p>
        <ul><li>get_powershell_execution_policy: powershell -executionpolicy 옵션으로 정책 매개변수가 포함된 프로세스 이벤트를 찾습니다.</li>
        <li>get_commandline_password: commandline에서 password를 사용하는 계정을 나열합니다. 안전하지 않은 자격증명은 공격에 악용될 소지가 있습니다.</li>
        <li>get_webshell_detection: IIS(w3wp.exe), nginx, Apache 등에서 httpd.exe, visualvnserver.exe등 일반적으로 시작하지 않는 의심스러운 프로세스를 찾습니다.</li></ul>
        <p>ProcessCommandLine 칼럼을 검토하십시오. 필요하다면 Sep1, 2로 되돌아가 재분석을 수행하도록 합니다.</p>'''))
        display(rst2)
    else:
        md("Not relevant alerts were found in your MDE logs, try expanding your timeframe")

Warning - query did not return any results.
No MDE Query Events, If you think that this is not correct, plz check the time range or FieName you are using
Warning - query did not return any results.
No MDE Query Events, If you think that this is not correct, plz check the time range or FieName you are using
Warning - query did not return any results.
No MDE Query Events, If you think that this is not correct, plz check the time range or FieName you are using


In [9]:
day_time = datetime.utcnow() - q_times.start
qry = f"""let KnowExploitesVulnsCISA = externaldata(cveID: string, vendorProject: string, product: string, vulnerabilityName: string, dateAdded: datetime, shortDescription: string, requiredAction: string, dueDate: datetime, 
notes: string)
[@"https://www.cisa.gov/sites/default/files/csv/known_exploited_vulnerabilities.csv"] 
with (format="csv", ignoreFirstRecord=True);
DeviceTvmSoftwareVulnerabilities
| join KnowExploitesVulnsCISA on $left.CveId == $right.cveID
| summarize
     TotalVulnerableDevices = count(),
     DeviceList = make_set(DeviceName),
     Description = make_set(shortDescription)
     by cveID
| sort by TotalVulnerableDevices
""" 
rst = md_prov.exec_query(query=qry)

if isinstance(rst, pd.DataFrame) and not rst.empty:
    display(HTML('''<h2>CISA 알려진 공격 취약성 데이터(CVE List)를 참조하여 MDE 취약점 결과를 표시합니다.</h2>
                 <p>CISA 취약점 카탈로그 다음 링크를 참고하였습니다. https://www.cisa.gov/known-exploited-vulnerabilities-catalog</p>'''))
    display(rst)

md("추가 분석을 진행할 Device를 선택해 주세요")
cveDeviceName = rst['DeviceList']
pick_shell = widgets.Dropdown(options=cveDeviceName,description="DeviceName",disabled=False)
pick_shell


,cveID,TotalVulnerableDevices,DeviceList,Description
0,CVE-2012-1823,1,[vm-honey],"[sapi/cgi/cgi_main.c in PHP, when configured as a CGI script, does not properly handle query str..."


Dropdown(description='DeviceName', options=(['vm-honey'],), value=['vm-honey'])

In [10]:
pick_shell.value

['vm-honey']

In [11]:
day_time = datetime.utcnow() - q_times.start
qry = f"""DeviceTvmSoftwareVulnerabilities|where DeviceName =
| summarize
     TotalVulnerableDevices = count(),
     DeviceList = make_set(DeviceName),
     Description = make_set(shortDescription)
     by cveID
| sort by TotalVulnerableDevices
""" 
rst = md_prov.exec_query(query=qry)
display(HTML('''<h2>CISA 알려진 공격 취약성 데이터(CVE List)를 참조하여 MDE 취약점 결과를 표시합니다.</h2>
<p>CISA 취약점 카탈로그 다음 링크를 참고하였습니다. https://www.cisa.gov/known-exploited-vulnerabilities-catalog</p>'''))
display(rst)

Missing expression. Fix syntax errors in your query.


HTTPStatusError: Client error '400 Bad Request' for url 'https://api.securitycenter.microsoft.com/api/advancedqueries/run'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/400

In [10]:
# Microsoft Defender for Endpoint TVM을 활용한 취약성 진단 가이드 노트북입니다.



# DeviceName, Time 필터링 값을 기준으로 CVE 취약점 정보를 표시합니다.
cvelist = Filter_huntingQuery.get_Device_vernervility(Seleted_Device.value, q_times.start, q_times.end)
cveresult = md_prov.exec_query(query=cvelist)
cveresult

,DeviceName,SoftwareName,SoftwareVersion,CveId,VulnerabilitySeverityLevel,count_
0,targetvm-win,windows_server_2022,10.0.20348.2159,CVE-2022-35737,High,1
1,targetvm-win,windows_server_2022,10.0.20348.2159,CVE-2024-20652,High,1
2,targetvm-win,windows_server_2022,10.0.20348.2159,CVE-2024-20653,High,1
3,targetvm-win,windows_server_2022,10.0.20348.2159,CVE-2024-20654,High,1
4,targetvm-win,windows_server_2022,10.0.20348.2159,CVE-2024-20655,Medium,1
5,targetvm-win,windows_server_2022,10.0.20348.2159,CVE-2024-20657,High,1
6,targetvm-win,windows_server_2022,10.0.20348.2159,CVE-2024-20658,High,1
7,targetvm-win,windows_server_2022,10.0.20348.2159,CVE-2024-20660,Medium,1
8,targetvm-win,windows_server_2022,10.0.20348.2159,CVE-2024-20661,High,1
9,targetvm-win,windows_server_2022,10.0.20348.2159,CVE-2024-20662,Medium,1


In [ ]:
# DeviceName 필터링 값을 기준으로 lsass.exe 자격 증명 액세스 여부를 헌팅합니다.
lsass = Filter_huntingQuery.get_Device_lsass_logon(Seleted_Device.value)
result = md_prov.exec_query(query=lsass)
result

In [63]:
# Auto_huntingQuery에 저장된 모든 헌팅 쿼리를 실행합니다.
hunting_numbers = [] 
for name in dir(Auto_huntingQuery):
    elt = getattr(Auto_huntingQuery, name)
    if type(elt) == types.FunctionType:
         hunting_numbers.append(elt.__name__)

for i in range(3):
    resutl1 = getattr(Auto_huntingQuery, hunting_numbers[i])() # huntingQuery.get_Detect_test()와 동일
    rst2 = md_prov.exec_query(query=resutl1)
    if len (rst2) == 0:
        display("데이터 결과가 없습니다.")
    else:
        display("데이터 쿼리 성공", rst2)

'데이터 쿼리 성공'

,Timestamp,DeviceId,DeviceName,ActionType,FileName,FolderPath,SHA1,SHA256,MD5,FileSize,AccountDomain,AccountName,AccountSid,RemoteUrl,RemoteDeviceName,ProcessId,ProcessCommandLine,ProcessCreationTime,ProcessTokenElevation,LogonId,RegistryKey,RegistryValueName,RegistryValueData,RemoteIP,RemotePort,...,InitiatingProcessMD5,InitiatingProcessFileName,InitiatingProcessFileSize,InitiatingProcessFolderPath,InitiatingProcessId,InitiatingProcessCommandLine,InitiatingProcessCreationTime,InitiatingProcessAccountDomain,InitiatingProcessAccountName,InitiatingProcessAccountSid,InitiatingProcessAccountUpn,InitiatingProcessAccountObjectId,InitiatingProcessVersionInfoCompanyName,InitiatingProcessVersionInfoProductName,InitiatingProcessVersionInfoProductVersion,InitiatingProcessVersionInfoInternalFileName,InitiatingProcessVersionInfoOriginalFileName,InitiatingProcessVersionInfoFileDescription,InitiatingProcessParentId,InitiatingProcessParentFileName,InitiatingProcessParentCreationTime,InitiatingProcessLogonId,ReportId,AppGuardContainerId,AdditionalFields
0,2023-12-22 01:13:36.378265+00:00,6d66cc22b2a8eefeb32bd9561fc31ca95d0a0286,test-ubuntu-2024.4n24mthkpnjuvl15wp44oenocg.syx.internal.cloudapp.net,ScriptContent,,,,a3c1d88980d5b3134d502da4708a6ecbbaafb19c8e4b9f5eeb75693da84eec53,,None,,,,,,None,,NaT,,None,,,,,None,...,,,None,,4049,,2023-12-22 01:13:32.607858+00:00,,,,,,,,,,,,0,,NaT,0,89,,"{""ScriptContent"":""#!/bin/sh\n\n# Script to concatenate rules files found in a base audit rules d..."
1,2023-12-22 01:13:36.669310+00:00,6d66cc22b2a8eefeb32bd9561fc31ca95d0a0286,test-ubuntu-2024.4n24mthkpnjuvl15wp44oenocg.syx.internal.cloudapp.net,ScriptContent,,,,7966a978b1b11eba4027dfabb58c5d32f7bcc389738c64a5a39e8213dbc8959c,,None,,,,,,None,,NaT,,None,,,,,None,...,,,None,,4076,,2023-12-22 01:13:33.909001+00:00,,,,,,,,,,,,0,,NaT,0,100,,"{""ScriptContent"":""#!/bin/sh\n\n#################################################################..."


'데이터 쿼리 성공'

,DeviceId,DeviceName,OSPlatform,OSVersion,OSArchitecture,SoftwareVendor,SoftwareName,SoftwareVersion,CveId,VulnerabilitySeverityLevel,RecommendedSecurityUpdate,RecommendedSecurityUpdateId,CveTags,CveMitigationStatus
0,46020bdd003b8875b53ad5068b47111c663cefd7,win-vm,WindowsServer2016,10.0.14393.5427,x64,microsoft,.net_framework,4.7.2.0,CVE-2023-21722,Medium,February 2023 Security Updates,5022838,[],
1,46020bdd003b8875b53ad5068b47111c663cefd7,win-vm,WindowsServer2016,10.0.14393.5427,x64,microsoft,.net_framework,4.7.2.0,CVE-2023-21808,High,February 2023 Security Updates,5022838,[],


Warning - query did not return any results.


'데이터 쿼리 성공'

{'Stats': {'ExecutionTime': 0.0156264,
  'resource_usage': {'cache': {'memory': {'hits': 0, 'misses': 0, 'total': 0},
    'disk': {'hits': 0, 'misses': 0, 'total': 0}},
   'cpu': {'user': '00:00:00', 'kernel': '00:00:00', 'total cpu': '00:00:00'},
   'memory': {'peak_per_node': 3670784}},
  'dataset_statistics': [{'table_row_count': 0, 'table_size': 0}]},
 'Schema': [{'Name': 'DeviceName', 'Type': 'String'},
  {'Name': 'InitiatingProcessAccountName', 'Type': 'String'},
  {'Name': 'InitiatingProcessCommandLine', 'Type': 'String'},
  {'Name': 'InitiatingProcessFileName', 'Type': 'String'},
  {'Name': 'InitiatingProcessFolderPath', 'Type': 'String'}],
 'Results': []}

In [58]:
rr = Auto_huntingQuery.get_test2()
aa = md_prov.exec_query(query=rr)
#print(dir(aa))
#if aa.empty:
if len (aa) == 0:    
    display("Data empty")
else:
    display("Data success", aa)


'Data success'

,DeviceId,DeviceName,OSPlatform,OSVersion,OSArchitecture,SoftwareVendor,SoftwareName,SoftwareVersion,CveId,VulnerabilitySeverityLevel,RecommendedSecurityUpdate,RecommendedSecurityUpdateId,CveTags,CveMitigationStatus
0,46020bdd003b8875b53ad5068b47111c663cefd7,win-vm,WindowsServer2016,10.0.14393.5427,x64,microsoft,.net_framework,4.7.2.0,CVE-2023-21722,Medium,February 2023 Security Updates,5022838,[],
1,46020bdd003b8875b53ad5068b47111c663cefd7,win-vm,WindowsServer2016,10.0.14393.5427,x64,microsoft,.net_framework,4.7.2.0,CVE-2023-21808,High,February 2023 Security Updates,5022838,[],


In [32]:
# Auto_huntingQuery에 저장된 모든 헌팅 쿼리를 실행합니다.
hunting_numbers = [] 
for name in dir(Auto_huntingQuery):
    elt = getattr(Auto_huntingQuery, name)
    if type(elt) == types.FunctionType:
         hunting_numbers.append(elt.__name__)

for i in range(3):
    resutl1 = getattr(Auto_huntingQuery, hunting_numbers[i])() # huntingQuery.get_Detect_test()와 동일
    rst2 = md_prov.exec_query(query=resutl1)
    if rst2.values[0] is not None:
        display(rst2)
    else:
        display("쿼리 결과가 없습니다.")

,Timestamp,DeviceId,DeviceName,ActionType,FileName,FolderPath,SHA1,SHA256,MD5,FileSize,AccountDomain,AccountName,AccountSid,RemoteUrl,RemoteDeviceName,ProcessId,ProcessCommandLine,ProcessCreationTime,ProcessTokenElevation,LogonId,RegistryKey,RegistryValueName,RegistryValueData,RemoteIP,RemotePort,...,InitiatingProcessMD5,InitiatingProcessFileName,InitiatingProcessFileSize,InitiatingProcessFolderPath,InitiatingProcessId,InitiatingProcessCommandLine,InitiatingProcessCreationTime,InitiatingProcessAccountDomain,InitiatingProcessAccountName,InitiatingProcessAccountSid,InitiatingProcessAccountUpn,InitiatingProcessAccountObjectId,InitiatingProcessVersionInfoCompanyName,InitiatingProcessVersionInfoProductName,InitiatingProcessVersionInfoProductVersion,InitiatingProcessVersionInfoInternalFileName,InitiatingProcessVersionInfoOriginalFileName,InitiatingProcessVersionInfoFileDescription,InitiatingProcessParentId,InitiatingProcessParentFileName,InitiatingProcessParentCreationTime,InitiatingProcessLogonId,ReportId,AppGuardContainerId,AdditionalFields
0,2023-12-12 06:05:49.868932+00:00,0e42acf9e7b3db64a6ec0edfec98f2e05966be5d,benchmark-mysql-cloocus.0402dunhed4erkwwz5httbatsh.syx.internal.cloudapp.net,ScriptContent,,,,a3c1d88980d5b3134d502da4708a6ecbbaafb19c8e4b9f5eeb75693da84eec53,,None,,,,,,None,,NaT,,None,,,,,None,...,,,None,,6338,,2023-12-12 06:05:49.501298+00:00,,,,,,,,,,,,0,,NaT,0,25,,"{""ScriptContent"":""#!/bin/sh\n\n# Script to concatenate rules files found in a base audit rules d..."
1,2023-12-12 06:05:49.974769+00:00,0e42acf9e7b3db64a6ec0edfec98f2e05966be5d,benchmark-mysql-cloocus.0402dunhed4erkwwz5httbatsh.syx.internal.cloudapp.net,ScriptContent,,,,7966a978b1b11eba4027dfabb58c5d32f7bcc389738c64a5a39e8213dbc8959c,,None,,,,,,None,,NaT,,None,,,,,None,...,,,None,,6362,,2023-12-12 06:05:49.613093+00:00,,,,,,,,,,,,0,,NaT,0,47,,"{""ScriptContent"":""#!/bin/sh\n\n#################################################################..."


,DeviceId,DeviceName,OSPlatform,OSVersion,OSArchitecture,SoftwareVendor,SoftwareName,SoftwareVersion,CveId,VulnerabilitySeverityLevel,RecommendedSecurityUpdate,RecommendedSecurityUpdateId,CveTags,CveMitigationStatus
0,46020bdd003b8875b53ad5068b47111c663cefd7,win-vm,WindowsServer2016,10.0.14393.5427,x64,microsoft,.net_framework,4.7.2.0,CVE-2023-21722,Medium,February 2023 Security Updates,5022838,[],
1,46020bdd003b8875b53ad5068b47111c663cefd7,win-vm,WindowsServer2016,10.0.14393.5427,x64,microsoft,.net_framework,4.7.2.0,CVE-2023-21808,High,February 2023 Security Updates,5022838,[],


Warning - query did not return any results.


TypeError: 'builtin_function_or_method' object is not subscriptable

In [249]:
from datetime import datetime, timedelta
class MyAlert:
    pass
md("Step1: 시간 필터링을 선택해 주세요")
alert = MyAlert()
alert.TimeGenerated = datetime.utcnow() - timedelta(1) # !!Office365 시간 필터는 24시간까지 제한이고, -7days 이상을 넘기면 안됩니다.  # type: ignore
q_times = msticpy.nbwidgets.QueryTime(units='day', max_before=20, before=3, max_after=1, origin_time=alert.TimeGenerated, auto_display=True)  # type: ignore


In [250]:
md("Step2: hostname을 선택해 주세요")
results = md_prov.MDATP.list_alerts(start=q_times.start, end=q_times.end)
Security_Query = results.DeviceName
Seleted_hostname = [i if i else "N/A" for i in Security_Query]
Seleted_hostname = [i for i in Seleted_hostname if i[0]]
Seleted_hostname = widgets.Dropdown(options = Seleted_hostname, description='hostname:')
display(Seleted_hostname)


Dropdown(description='hostname:', options=('vm-honey', 'win-vm', 'test-bastion-wi', 'eus-win-vm', 'test-tar-wi…

In [274]:
msticpy.query_help()

AttributeError: module 'msticpy' has no attribute 'query_help'

In [11]:
# reference:https://msticpy.readthedocs.io/en/latest/data_acquisition/DataQueries.html#queries-for-microsoft-365-defender
md("step3: 분석을 수행할 쿼리를 선택해 주세요. 쿼리에 대한 설명은 하단에 있습니다.")
rst = md_prov.browse_queries()
#aa = md_prov.query_help(rst.value) # 도움말만 출력
rst

In [ ]:
ips = '162.244.80.235'

results_df = msticpy.TILookup.result_to_df(
    ti_lookup.lookup_ioc(ips)
)
results_df